In [5]:
%%writefile ass2parallelmergesort.cu

#include <iostream>
#include <omp.h>
using namespace std;

void merge(int arr[], int low, int mid, int high) {
    // Create arrays of left and right partitions
    int n1 = mid - low + 1;
    int n2 = high - mid;
    int left[n1];
    int right[n2];

    // Copy all left elements
    for (int i = 0; i < n1; i++)
        left[i] = arr[low + i];

    // Copy all right elements
    for (int j = 0; j < n2; j++)
        right[j] = arr[mid + 1 + j];

    // Compare and place elements
    int i = 0, j = 0, k = low;
    while (i < n1 && j < n2) {
        if (left[i] <= right[j]) {
            arr[k] = left[i];
            i++;
        } else {
            arr[k] = right[j];
            j++;
        }
        k++;
    }

    // If any elements are left out
    while (i < n1) {
        arr[k] = left[i];
        i++;
        k++;
    }
    while (j < n2) {
        arr[k] = right[j];
        j++;
        k++;
    }
}

void parallelMergeSort(int arr[], int low, int high) {
    if (low < high) {
        int mid = (low + high) / 2;
#pragma omp parallel sections
        {
#pragma omp section
            {
                parallelMergeSort(arr, low, mid);
            }
#pragma omp section
            {
                parallelMergeSort(arr, mid + 1, high);
            }
        }
        merge(arr, low, mid, high);
    }
}

void mergeSort(int arr[], int low, int high) {
    if (low < high) {
        int mid = (low + high) / 2;
        mergeSort(arr, low, mid);
        mergeSort(arr, mid + 1, high);
        merge(arr, low, mid, high);
    }
}

int main() {
    int n = 1000;
    int arr[n];
    double start_time, end_time;

    // Create an array with numbers starting from n to 1.
    for (int i = 0, j = n; i < n; i++, j--)
        arr[i] = j;

    // Measure Sequential Time
    start_time = omp_get_wtime();
    mergeSort(arr, 0, n - 1);
    end_time = omp_get_wtime();
    cout << "Time taken by sequential algorithm: " << end_time - start_time << " seconds\n";

    // Reset the array
    for (int i = 0, j = n; i < n; i++, j--)
        arr[i] = j;

    // Measure Parallel time
    start_time = omp_get_wtime();
    parallelMergeSort(arr, 0, n - 1);
    end_time = omp_get_wtime();
    cout << "Time taken by parallel algorithm: " << end_time - start_time << " seconds";

    return 0;
}

Writing ass2parallelbubble.cu


In [6]:
!nvcc -o ass2parallelbubble ass2parallelbubble.cu -Xcompiler -fopenmp
!./ass2parallelbubble

ass2parallelbubble.cu(42): warning #177-D: variable "brr" was declared but never referenced
  int brr[n];
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Sequential Bubble Sort took : 1.549e-06 seconds.
1 2 3 4 5 6 7 8 9 10 
Parallel Bubble Sort took : 2.76e-06 seconds.
1 2 3 4 5 6 7 8 9 10 
